# Mock Lambda Function

In [7]:
test_comment = "The focus on raising the age of gun buyers to 21 is not random: Individuals ages 18 to 20 commit homicides using guns at a rate that’s four times higher than the rate for those over 21, according to Everytown for Gun Safety, using F.B.I. and census data.\"<br/><br/>No, it may not be RANDOM, but this statement is highly DECEPTIVE. The overwhelming majority of homicides are committed with illegal handguns. Those who possess them are not purchasing them at retail stores using credit cards and submitting to NICS background checks. They are most often young, frequently gang members, and associated with the drug trade.<br/>In virtually all cases, statistics from Everytown can be shown to be distortions of legitimate FBI, DoJ, and census data."

test_event = {'comment': test_comment}
test_context = None

In [8]:
import boto3

s3 = boto3.resource('s3')
obj = s3.Object('sagemaker-us-east-1-228889150161','data/nyt-features/vocab.json')
obj.download_file('vocab.json')

In [9]:
import os 
os.environ['ENDPOINT'] = 'ntm-nyt'

In [10]:
import json 
import re 
from collections import Counter
import boto3
import io
import os


def load_vocab():
    with open('vocab.json', 'r') as json_file:
        return json.load(json_file)

def mock_function(event, context):
    vocab = load_vocab()
    tokens = re.split(r'\W+', event['comment'])
    
    lowercase = map(lambda token: token.lower(), tokens)
    vocab_words = filter(lambda token: token in vocab, lowercase)
    idx = map(lambda token: vocab[token], vocab_words)
    counts = Counter(idx)
    
    #https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html
    keys = sorted(counts.keys())
    values = list(map(lambda key: counts[key], keys))
    features = { "keys": keys, "shape": [len(vocab)],"values": values }
    
    ntm_input = {"instances":[{"data": {"features": features}}]}
    req_body = json.dumps(ntm_input)
    print(req_body)
    
    client = boto3.client('sagemaker-runtime')
    resp = client.invoke_endpoint(
        EndpointName=os.environ['ENDPOINT'],
        Body=req_body.encode('utf-8'),
        ContentType='application/json',
        Accept='application/json'
    )
        
    predictions = json.load(resp["Body"])
    return predictions

In [11]:
response = mock_function(test_event, test_context)

{"instances": [{"data": {"features": {"keys": [26, 27, 77, 80, 102, 158, 238, 243, 246, 317, 323, 410, 468, 471, 550, 687, 738, 786, 798, 846, 869, 912, 961, 1058, 1070, 1081, 1128, 1200, 1243, 1270, 1363, 1851, 1852, 2137, 2173, 2295, 2527, 2559, 2638, 2765, 2828, 3137, 3449, 3690, 4185, 4417, 4599, 4854], "shape": [5000], "values": [1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]}}}]}


In [12]:
print(response)

{'predictions': [{'topic_weights': [0.0234206449, 0.0219569486, 0.0244625956, 0.0242984146, 0.0238350872, 0.0264372826, 0.0243257564, 0.0230258852, 0.1415586621, 0.0268683769, 0.0247516725, 0.3408972621, 0.0275082532, 0.0235611908, 0.0246553216, 0.1024556607, 0.0240754802, 0.0239798501, 0.0235598832, 0.0243657827]}]}


In [15]:
import numpy as np
weigths = response['predictions'][0]['topic_weights']
w = np.asarray(weigths)
np.argmax(w, axis=0)

11

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
sagemaker_session.delete_endpoint('ntm-nyt')